## IMPORTAR LAS LIBRERIAS

Actualizar a las que se usen finalmente en tu proyecto.

In [63]:
import numpy as np
import pandas as pd
import cloudpickle

#Automcompletar rápido
%config IPCompleter.greedy=True

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler


from xgboost import XGBClassifier

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [64]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## CARGAR LOS DATOS

### Ruta del proyecto

In [65]:
ruta_proyecto = 'C:/Users/USER/Desktop/DSD4B (ZERO TO HERO)/03_MACHINE_LEARNING/08_CASOS/01_LEADSCORING'

### Nombre del fichero de datos

In [66]:
nombre_fichero_datos = 'leads_2020_2022.csv'

### Cargar los datos

In [67]:
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos

df = pd.read_csv(ruta_completa,index_col=0,sep=';')

### Seleccionar solo las variables finales

#### Cargar la lista de variables finales

In [68]:
nombre_variables_finales = ruta_proyecto + '/05_Resultados/' + 'variables_finales.pickle'

pd.read_pickle(nombre_variables_finales).sort_index().index.to_list()

['ambito_Agentes de Carga',
 'ambito_Exportadores',
 'ambito_Null',
 'fuente_Organic Search',
 'ocupacion_Gerente de operaciones logisticas',
 'origen_Landing Page Submission',
 'paginas_vistas_visita_mms',
 'score_actividad_mms',
 'score_perfil_mms',
 'tiempo_en_site_total_mms',
 'ult_actividad_Chat Conversation',
 'ult_actividad_Converted to Lead',
 'ult_actividad_Page Visited on Website',
 'ult_actividad_SMS Sent',
 'visitas_total_mms']

#### Apuntar (manualmente) la lista de variables finales sin extensiones

In [69]:
variables_finales = ['ambito',
                   'fuente',
                   'ocupacion',
                   'origen',
                   'paginas_vistas_visita',
                   'score_actividad',
                   'score_perfil',
                   'tiempo_en_site_total',
                   'ult_actividad',
                   'visitas_total'
                  ]

#### Crear la matriz de variables procesos (excel)

Ir a la plantilla de Excel "Fase Producción Plantilla Procesos" y crear la matriz de variables por procesos.

#### Actualizar las importaciones

Ir arriba a la celda de importacion de paquetes y actualizarlos con los que finalmente vamos a usar.

## ESTRUCTURA DE LOS DATASETS

### Eliminar registros

In [70]:
df.shape

(6810, 23)

#### Por duplicados

In [71]:
df.drop_duplicates(inplace = True)

In [72]:
df.shape

(6785, 23)

#### Por EDA

In [73]:
df=df.loc[(df.no_llamar != 'Otros') & (df.no_enviar_email != 'Yes') & (df.ult_actividad != 'Email Bounced')]

#### Para x

Quedarse solo con las de la lista.

In [74]:
x = df[variables_finales].copy()

In [75]:
x.shape

(6232, 10)

#### Para y

Especificar la target.

In [76]:
target = 'compra'

Crear el y.

In [77]:
y = df[target].copy()

## CREAR EL PIPELINE

### Instanciar calidad de datos

#### Crear la función

In [78]:
def calidad_datos(df):
    temp = df.astype({'visitas_total': 'Int64'})
    
    temp['ambito'] = temp['ambito'].replace({
    'SG1A – Agentes de Carga': 'Agentes de Carga',
    'SG2A - Importadores': 'Importadores',
    'SG2B - Exportadores': 'Exportadores',
    'SG3 - Otros': 'Otros',
    'SG1C – Aerolíneas': 'Aerolineas'
    })
    
    def imputar_moda(variable):
        return(variable.fillna(variable.mode()[0]))

    var_imputar_moda = ['ocupacion']
    
    temp[var_imputar_moda] = temp[var_imputar_moda].apply(imputar_moda)
    
    
    var_imputar_valor = ['origen','fuente', 'ult_actividad',
                     'ambito']
    valor = 'Null'
    
    temp[var_imputar_valor] = temp[var_imputar_valor].fillna(valor)
    
    
    var_imputar_mediana = ['paginas_vistas_visita',
                     'score_actividad',
                     'score_perfil',
                     'tiempo_en_site_total',
                     'visitas_total']
    
    def imputar_mediana(variable):
        if pd.api.types.is_integer_dtype(variable):
            return(variable.fillna(int(variable.median())))
        else:
            return(variable.fillna(variable.median()))

    temp[var_imputar_mediana] = temp[var_imputar_mediana].apply(imputar_mediana)
    
    def agrupar_cat_raras(variable, criterio = 0.05):
        #Calcula las frecuencias
        frecuencias = variable.value_counts(normalize=True)
        #Identifica las que están por debajo del criterio
        temp = [cada for cada in frecuencias.loc[frecuencias < criterio].index.values]
        #Las recodifica en 'Otros'
        temp2 = np.where(variable.isin(temp),'Otros',variable)
        #Devuelve el resultado
        return(temp2)

    var_agrupar_cat_rars = ['ocupacion','ambito','origen','fuente','ult_actividad']
    
    for variable in var_imputar_moda:
        temp[variable] = agrupar_cat_raras(temp[variable],criterio = 0.02)
    
    
    temp['visitas_total']= temp['visitas_total'].clip(0,50)
    temp['paginas_vistas_visita']= temp['paginas_vistas_visita'].clip(0,20)
    
    return(temp)

In [79]:
calidad_datos(x)

,ambito,fuente,ocupacion,origen,paginas_vistas_visita,score_actividad,score_perfil,tiempo_en_site_total,ult_actividad,visitas_total
id,,,,,,,,,,
660737,SG1A - Agentes de Carga,Chat,Asistente de logistica,API,0.0,15.0,15.0,0,Page Visited on Website,0
660728,SG1A - Agentes de Carga,Organic Search,Asistente de logistica,API,2.5,15.0,15.0,674,Email Opened,5
660680,Null,Chat,Asistente de logistica,API,0.0,17.0,15.0,0,Chat Conversation,0
660673,Importadores,Google,Asistente de logistica,Landing Page Submission,2.0,14.0,20.0,1640,Email Opened,2
660562,SG1A - Agentes de Carga,Organic Search,Asistente de logistica,API,11.0,16.0,16.0,1538,Email Opened,11
...,...,...,...,...,...,...,...,...,...,...
579701,Importadores,Google,Asistente de logistica,Landing Page Submission,2.0,13.0,20.0,870,Email Opened,2
579642,Importadores,Direct Traffic,Asistente de logistica,Landing Page Submission,2.0,14.0,20.0,1770,SMS Sent,2
579622,SG1A - Agentes de Carga,Direct Traffic,Asistente de logistica,API,2.6,14.0,16.0,1409,SMS Sent,13


#### Convertirla en transformer

In [80]:
hacer_calidad_datos = FunctionTransformer(calidad_datos)

### Instanciar transformación de variables

In [81]:
var_ohe = ['ocupacion','ambito','origen','fuente','ult_actividad']
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

var_mms = ['paginas_vistas_visita','score_actividad','score_perfil','tiempo_en_site_total','visitas_total']
mms = MinMaxScaler()

### Crear el pipe del preprocesamiento

#### Crear el column transformer

In [82]:
ct = make_column_transformer(
    (ohe, var_ohe),
    (mms, var_mms),
    remainder='drop')

#### Crear el pipeline del preprocesamiento

In [83]:
pipe_prepro = make_pipeline(hacer_calidad_datos, 
                            ct)

### Instanciar el modelo

#### Instanciar el algoritmo

In [84]:
modelo = XGBClassifier(n_jobs = -1, 
                       verbosity = 0,
                       learning_rate = 0.05,
                       max_depth = 5,
                       reg_alpha = 1,
                       reg_lambda = 1,
                       n_estimators = 100,
                       use_label_encoder=False
                       )

#### Crear el pipe final de entrenamiento

In [85]:
pipe_entrenamiento = make_pipeline(pipe_prepro,modelo)

#### Guardar el pipe final de entrenamiento

In [86]:
nombre_pipe_entrenamiento = 'pipe_entrenamiento.pickle'

ruta_pipe_entrenamiento = ruta_proyecto + '/04_Modelos/' + nombre_pipe_entrenamiento

with open(ruta_pipe_entrenamiento, mode='wb') as file:
   cloudpickle.dump(pipe_entrenamiento, file)

#### Entrenar el pipe final de ejecución

In [87]:
pipe_ejecucion = pipe_entrenamiento.fit(x,y)

## GUARDAR EL PIPE

### Nombre del pipe final de ejecución

In [88]:
nombre_pipe_ejecucion = 'pipe_ejecucion.pickle'

### Guardar el pipe final de ejecución

In [89]:
ruta_pipe_ejecucion = ruta_proyecto + '/04_Modelos/' + nombre_pipe_ejecucion

with open(ruta_pipe_ejecucion, mode='wb') as file:
   cloudpickle.dump(pipe_ejecucion, file)